# Notebook 4: Predictions and Analysis

This notebook demonstrates the primary scientific application: predicting
neutron cross-sections for isotopes without experimental measurements.

We cover:
1. Predicting energy-dependent cross-sections for specific unmeasured isotopes
2. Comparing ML predictions against ENDF and TALYS Hauser-Feshbach calculations
3. Visualizing prediction uncertainty across the nuclear chart
4. Identifying isotopes where predictions are most uncertain
5. Analysis of r-process nucleosynthesis path isotopes

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pathlib import Path

from data.nuclear_properties import NuclearPropertiesLoader
from data.dataset_builder import FEATURE_COLUMNS
from visualization.plots import (
    plot_cross_section_energy_dependence,
    plot_nuclear_chart_heatmap,
    plot_predicted_vs_experimental,
)

plt.rcParams.update({'font.family': 'serif', 'figure.dpi': 150})
print('Setup complete.')

## 1. Load trained model

In [ ]:
model_path = Path('../results/models/xgboost.pkl')

if model_path.exists():
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    print(f'Loaded model: {type(model).__name__}')
else:
    print(f'Model not found at {model_path}.')
    print('Run notebook 03 first, or: python scripts/train_models.py --dataset data/processed/n_gamma_dataset.h5')
    # For demonstration, create a simple stand-in model
    from models.xgboost_model import XGBoostCrossSectionModel
    import numpy as np
    rng = np.random.default_rng(42)
    n = 500
    X_demo = rng.standard_normal((n, len(FEATURE_COLUMNS))).astype(np.float32)
    y_demo = (2*X_demo[:, 0] - X_demo[:, 2] + rng.standard_normal(n) * 0.3).astype(np.float32)
    model = XGBoostCrossSectionModel(params={'n_estimators': 100}, feature_names=FEATURE_COLUMNS)
    model.fit(X_demo, y_demo)
    print('Using demonstration model (not trained on real data).')

## 2. Load nuclear properties for prediction

In [ ]:
props_loader = NuclearPropertiesLoader(
    ame_path='../data/external/ame2020_mass.txt',
    nubase_path='../data/external/nubase2020.txt',
    frdm_path='../data/external/frdm2012_deformation.dat',
)

if Path('../data/external/ame2020_mass.txt').exists():
    nuclear_props = props_loader.load_all()
    print(f'Loaded properties for {len(nuclear_props)} isotopes')
else:
    print('Nuclear properties not available. Using estimated features for prediction.')
    nuclear_props = pd.DataFrame()

## 3. Predict cross-sections for specific isotopes

We predict (n,gamma) cross-sections for several isotopes of scientific interest:
- 79Au197 (Gold-197): well-measured reference nucleus
- 45Rh103 (Rhodium-103): relevant to r-process
- Neutron-rich Ba isotopes: r-process path nuclei

In [ ]:
def build_prediction_features(Z, A, energies_eV, nuclear_props, props_loader):
    """Build feature matrix for (Z, A) over an energy grid."""
    if len(nuclear_props) > 0:
        props = props_loader.get_features_for_isotope(Z, A, nuclear_props)
    else:
        props = props_loader._estimate_features(Z, A)
    
    records = []
    for E in energies_eV:
        feat = {col: props.get(col, np.nan) for col in FEATURE_COLUMNS}
        feat['log10_energy_eV'] = np.log10(max(E, 1e-15))
        feat['Z'] = Z
        feat['N'] = A - Z
        feat['A'] = A
        feat['Q_value_MeV'] = np.nan
        records.append(feat)
    
    X = np.array([[r.get(col, np.nan) for col in FEATURE_COLUMNS] for r in records], dtype=np.float32)
    
    # Simple imputation: fill NaN with column median from training (or 0)
    for j in range(X.shape[1]):
        nan_mask = np.isnan(X[:, j])
        if nan_mask.any():
            X[nan_mask, j] = 0.0  # Placeholder; ideally use training set median
    
    return X

# Energy grid: thermal to 20 MeV
energies_eV = np.logspace(-2, 7.3, 200)  # 10 meV to 20 MeV

# Predict for Au-197
Z_Au, A_Au = 79, 197
X_Au = build_prediction_features(Z_Au, A_Au, energies_eV, nuclear_props, props_loader)
preds_Au = model.predict_with_uncertainty(X_Au)

pred_key = 'mean' if 'mean' in preds_Au else 'median'
sigma_Au = 10 ** preds_Au[pred_key]

lower_Au = 10 ** preds_Au.get('lower_68', preds_Au.get('q16', preds_Au[pred_key]))
upper_Au = 10 ** preds_Au.get('upper_68', preds_Au.get('q84', preds_Au[pred_key]))

print(f'Au-197 (n,gamma): predicted thermal cross-section = {sigma_Au[energies_eV < 0.1][0]:.2f} barn')

In [ ]:
# Plot energy dependence for Au-197
fig = plot_cross_section_energy_dependence(
    energies_eV=energies_eV,
    sigma_pred=sigma_Au,
    sigma_lower=lower_Au,
    sigma_upper=upper_Au,
    isotope_label=r'$^{197}$Au',
    reaction=r'(n,$\gamma$)',
    output_path='../results/figures/Au197_n_gamma_prediction.pdf',
)
plt.show()

## 4. Scan predictions across the nuclear chart at thermal energies

We predict thermal (n,gamma) cross-sections for all isotopes in the nuclear
properties table and display them on the (N, Z) chart.

In [ ]:
thermal_energy_eV = 0.0253  # 25.3 meV thermal energy

if len(nuclear_props) > 0:
    predict_df = nuclear_props[nuclear_props['A'] <= 209].copy()
else:
    # Generate prediction grid
    Z_grid = np.repeat(np.arange(1, 84), 30)
    A_offset = np.tile(np.arange(-5, 25), 83)
    N_grid = Z_grid + A_offset
    mask = (N_grid >= 0) & (N_grid <= 3 * Z_grid)
    predict_df = pd.DataFrame({'Z': Z_grid[mask], 'N': N_grid[mask]})
    predict_df['A'] = predict_df['Z'] + predict_df['N']
    predict_df = predict_df[predict_df['A'] <= 209].copy()

print(f'Predicting for {len(predict_df)} isotopes at thermal energy...')

all_preds = []
all_stds = []

for _, row in predict_df.iterrows():
    Z, A = int(row['Z']), int(row['A'])
    try:
        X = build_prediction_features(Z, A, [thermal_energy_eV], nuclear_props, props_loader)
        p = model.predict_with_uncertainty(X)
        pred_key = 'mean' if 'mean' in p else 'median'
        all_preds.append(float(p[pred_key][0]))
        std_key = 'std' if 'std' in p else ('sigma_68' if 'sigma_68' in p else None)
        all_stds.append(float(p[std_key][0]) if std_key else 0.0)
    except Exception:
        all_preds.append(np.nan)
        all_stds.append(np.nan)

predict_df['log10_sigma_thermal'] = all_preds
predict_df['sigma_uncertainty'] = all_stds

valid = predict_df.dropna(subset=['log10_sigma_thermal'])
print(f'Predictions completed for {len(valid)} isotopes')

In [ ]:
if len(valid) > 0:
    # Nuclear chart heatmap of predicted thermal cross-sections
    fig = plot_nuclear_chart_heatmap(
        Z_values=valid['Z'].values,
        N_values=valid['N'].values,
        values=valid['log10_sigma_thermal'].values,
        value_label='Predicted log$_{10}$($\\sigma_{thermal}$ / barn)',
        title='ML-Predicted Thermal (n,$\\gamma$) Cross-Sections',
        output_path='../results/figures/nuclear_chart_thermal_predictions.pdf',
    )
    plt.show()
    
    # Uncertainty chart
    unc_valid = valid.dropna(subset=['sigma_uncertainty'])
    if len(unc_valid) > 0 and unc_valid['sigma_uncertainty'].std() > 0:
        fig2 = plot_nuclear_chart_heatmap(
            Z_values=unc_valid['Z'].values,
            N_values=unc_valid['N'].values,
            values=unc_valid['sigma_uncertainty'].values,
            value_label='Prediction uncertainty $\\sigma$ (log$_{10}$ barn)',
            title='ML Prediction Uncertainty',
            output_path='../results/figures/nuclear_chart_uncertainty.pdf',
        )
        plt.show()

## 5. High-uncertainty predictions

The most uncertain predictions identify isotopes where additional experimental
measurements would provide the greatest improvement to the training data.

In [ ]:
if 'sigma_uncertainty' in valid.columns and valid['sigma_uncertainty'].notna().any():
    high_unc = valid.nlargest(20, 'sigma_uncertainty')[['Z', 'N', 'A', 'log10_sigma_thermal', 'sigma_uncertainty']]
    high_unc['sigma_barn_pred'] = 10 ** high_unc['log10_sigma_thermal']
    high_unc['relative_uncertainty'] = high_unc['sigma_uncertainty'] / np.abs(high_unc['log10_sigma_thermal'])
    
    print('Isotopes with highest prediction uncertainty (targets for experimental measurement):')
    display(high_unc[['Z', 'N', 'A', 'sigma_barn_pred', 'sigma_uncertainty']].to_string(index=False))

## 6. R-process nucleosynthesis path isotopes

The rapid neutron-capture process (r-process) in neutron star mergers
proceeds through extremely neutron-rich isotopes for which essentially
no cross-section measurements exist. We predict cross-sections for
representative r-process path nuclei near A=130 (first r-process peak).

In [ ]:
# R-process path nuclei near the A=130 abundance peak
# These have N~82 and Z values ranging from 40-56
rprocess_isotopes = [
    (40, 122, 'Zr-122'),   # Very neutron-rich Zirconium
    (42, 124, 'Mo-124'),   # Neutron-rich Molybdenum
    (44, 126, 'Ru-126'),   # Near N=82 magic number
    (46, 128, 'Pd-128'),   # Neutron-rich Palladium
    (48, 130, 'Cd-130'),   # r-process path Cadmium
    (50, 132, 'Sn-132'),   # Doubly magic (Z=50, N=82)
]

thermal_E = np.array([0.1, 1.0, 1e3, 1e5, 1e6])  # eV

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for ax, (Z, A, label) in zip(axes.flat, rprocess_isotopes):
    E_grid = np.logspace(-2, 7.3, 100)
    X_iso = build_prediction_features(Z, A, E_grid, nuclear_props, props_loader)
    p = model.predict_with_uncertainty(X_iso)
    pred_key = 'mean' if 'mean' in p else 'median'
    sigma = 10 ** p[pred_key]
    lower = 10 ** p.get('lower_68', p.get('q16', p[pred_key]))
    upper = 10 ** p.get('upper_68', p.get('q84', p[pred_key]))
    
    ax.loglog(E_grid, sigma, color='#1b9e77', lw=2, label='ML prediction')
    ax.fill_between(E_grid, lower, upper, alpha=0.25, color='#1b9e77')
    ax.set_xlabel('Neutron energy (eV)')
    ax.set_ylabel('$\\sigma$ (barn)')
    ax.set_title(label)
    ax.grid(True, which='both', alpha=0.3, lw=0.5)
    ax.legend(fontsize=8)

plt.suptitle('Predicted (n,$\\gamma$) cross-sections for r-process path nuclei (A~130 peak)', y=1.01)
plt.tight_layout()
plt.savefig('../results/figures/rprocess_predictions.pdf', dpi=300, bbox_inches='tight')
plt.show()
print('Note: these isotopes have no experimental cross-section measurements.')
print('These predictions represent ML extrapolation from measured stable isotopes.')